In [1]:
import re
import csv
from typing import List, Dict
import pandas as pd
import re
import csv
from typing import List

In [2]:
# read excell shet to df
import pandas as pd

path = "./Ocean Governance and ocean economy governance matrix_IGOs.xlsx"
data = pd.read_excel(path, sheet_name="Sheet1")

# backup
df = data.copy(deep=True)

In [3]:
df = df[["Institutions", "Defined inter-institutional Relationship"]]

In [ ]:
import pandas as pd
import re
import csv
from typing import List

# Step 1: Cleaning the Data
def clean_data(raw_text: str) -> str:
    """Clean raw text by removing typos, URLs, and redundant details."""
    if pd.isna(raw_text):
        return ""
    
    text = raw_text.replace("IIOC", "IOC").replace("collborates", "collaborates").replace("instututions", "institutions")
    text = re.sub(r'\(https?://[^\s)]+\)', '', text)
    text = text.replace("fosters relationships with various institutions to support its mission", "supports its mission through institutional relationships")
    text = text.replace("which are responsible for implementing the decisions and strategies", "tasked with implementing decisions")
    text = text.replace("United Nations", "UN").replace("member states", "Member States")
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Step 2: Summarizing the Data
def summarize_data(cleaned_text: str) -> str:
    """Summarize cleaned text into 1-2 sentences."""
    if not cleaned_text:
        return "No data provided."
    
    entities = re.findall(r'(UN|UNESCO|WMO|FAO|Member States|[A-Z]{2,})', cleaned_text)
    unique_entities = list(dict.fromkeys(entities))[:5]
    mechanisms = "mechanisms" if "agreement" in cleaned_text or "program" in cleaned_text else "frameworks"
    purpose = "research" if "research" in cleaned_text else "governance" if "governance" in cleaned_text else "development"
    
    if unique_entities:
        summary = f"The {unique_entities[0]} coordinates with {', '.join(unique_entities[1:])} through {mechanisms} to advance {purpose}."
    else:
        summary = "The organization coordinates through unspecified mechanisms to advance development."
    
    return summary

# Step 3: Creating Themes (Enhanced Version)
def assign_themes(cleaned_text: str, summary: str) -> List[str]:
    """Assign themes based on patterns in cleaned text and summary with scoring."""
    themes = set()
    
    # Define 10 key phrases from your methodology as evidence
    key_phrases = {
        "Collaboration with UN specialized agencies": ("Collaboration with External Entities", 2),
        "Multilateral cooperation and partnerships": ("Collaboration with External Entities", 1),
        "Joint research and development initiatives": ("Technical and Capacity-Building Support", 2),
        "Coordinated policy advocacy": ("Strategic Leadership and Oversight", 1),
        "Shared capacity building programs": ("Technical and Capacity-Building Support", 1),
        "Inter-agency agreements and MOUs": ("Multi-Level Governance Structures", 2),
        "Collaborative environmental monitoring and management": ("Technical and Capacity-Building Support", 1),
        "Exchange of knowledge and best practices": ("Technical and Capacity-Building Support", 1),
        "Co-funding and financial partnerships": ("Collaboration with External Entities", 1),
        "Networking and information exchange platforms": ("Collaboration with External Entities", 1)
    }
    
    # Scoring dictionary for themes
    theme_scores = {
        "Multi-Level Governance Structures": 0,
        "Collaboration with External Entities": 0,
        "Regional and Local Implementation": 0,
        "Technical and Capacity-Building Support": 0,
        "Strategic Leadership and Oversight": 0
    }
    
    # Analyze cleaned text for evidence
    cleaned_lower = cleaned_text.lower()
    summary_lower = summary.lower()
    combined_text = cleaned_lower + " " + summary_lower
    
    # Score based on key phrases
    for phrase, (theme, weight) in key_phrases.items():
        if phrase.lower() in combined_text:
            theme_scores[theme] += weight
    
    # Additional pattern checks
    if any(term in cleaned_lower for term in ["agreement", "convention", "statutes", "assembly", "council", "secretariat"]):
        theme_scores["Multi-Level Governance Structures"] += 2
    if any(term in cleaned_lower for term in ["UN ", "UNESCO", "WMO", "FAO", "IMO", "Member States"]):
        theme_scores["Collaboration with External Entities"] += 2
    if any(term in cleaned_lower for term in ["regional", "local", "national", "south-south", "countries"]):
        theme_scores["Regional and Local Implementation"] += 2
    if any(term in cleaned_lower for term in ["research", "technical", "capacity", "training", "support", "data"]):
        theme_scores["Technical and Capacity-Building Support"] += 2
    if any(term in summary_lower for term in ["align", "drive", "oversight", "strategy", "lead"]):
        theme_scores["Strategic Leadership and Oversight"] += 2
    
    # Assign themes based on score threshold (e.g., >= 2)
    for theme, score in theme_scores.items():
        if score >= 2:
            themes.add(theme)
    
    # Ensure at least one theme
    if not themes:
        themes.add("Collaboration with External Entities")
    
    return sorted(themes)  # Return sorted list for consistency

# Step 4: Constructing the Table
def process_column(df: pd.DataFrame, column_name: str, output_file: str = "inter_institutional_relationships.csv"):
    """Process the specified column in the DataFrame and output to CSV."""
    if column_name not in df.columns or "Institutions" not in df.columns:
        raise ValueError("DataFrame must contain 'Institution' and the specified column.")
    
    table = []
    for index, row in df.iterrows():
        institution = row["Institutions"]
        raw_text = row[column_name]
        
        cleaned = clean_data(raw_text)
        summary = summarize_data(cleaned)
        themes = assign_themes(cleaned, summary)
        
        table.append({
            "Institutions": institution,
            "Cleaned Practical Vertical Coordination": cleaned,
            "Summarized Vertical Coordination": summary,
            "Themes": ";".join(themes)
        })
    
    with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ["Institutions", "Cleaned Practical Vertical Coordination", "Summarized Vertical Coordination", "Themes"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        for row in table:
            writer.writerow(row)
    
    print(f"Processing complete. Output saved to {output_file}")

In [4]:
df.head(3)

,Institutions,Defined inter-institutional Relationship
0,Intergovernmental Oceanographic Commission (IOC),IOC collaborates with UN specialized agencies ...
1,Food and Agriculture Organization of the Unite...,"As stated in its constitution, the FAO maintai..."
2,Convention on the Intergovernmental Maritime C...,The IMO collaborates with a diverse array of o...


In [5]:
import pandas as pd
import re
import csv
from typing import List

# Step 1: Cleaning the Data
def clean_data(raw_text: str) -> str:
    """Clean raw text by removing typos, URLs, and redundant details."""
    if pd.isna(raw_text):
        return ""
    
    text = raw_text.replace("IIOC", "IOC").replace("collborates", "collaborates").replace("instututions", "institutions")
    text = re.sub(r'\(https?://[^\s)]+\)', '', text)
    text = text.replace("fosters relationships with various institutions to support its mission", "supports its mission through institutional relationships")
    text = text.replace("which are responsible for implementing the decisions and strategies", "tasked with implementing decisions")
    text = text.replace("United Nations", "UN").replace("member states", "Member States")
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Step 2: Summarizing the Data
def summarize_data(cleaned_text: str) -> str:
    """Summarize cleaned text into 1-2 sentences."""
    if not cleaned_text:
        return "No data provided."
    
    entities = re.findall(r'(UN|UNESCO|WMO|FAO|Member States|[A-Z]{2,})', cleaned_text)
    unique_entities = list(dict.fromkeys(entities))[:5]
    mechanisms = "mechanisms" if "agreement" in cleaned_text or "program" in cleaned_text else "frameworks"
    purpose = "research" if "research" in cleaned_text else "governance" if "governance" in cleaned_text else "development"
    
    if unique_entities:
        summary = f"The {unique_entities[0]} coordinates with {', '.join(unique_entities[1:])} through {mechanisms} to advance {purpose}."
    else:
        summary = "The organization coordinates through unspecified mechanisms to advance development."
    
    return summary

# Updated function for the new theme mapping
def assign_themes_updated(cleaned_text: str, summary: str) -> List[str]:
    """Assign new themes based on specific keywords and phrases."""
    themes = set()

    # Define new theme scores
    theme_scores = {
        "UN System Integration": 0, 
        "Regional Collaboration": 0, 
        "Scientific Partnerships": 0,
        "Regulatory Alignment": 0, 
        "Environmental Cooperation": 0, 
        "Economic Linkages": 0,
        "Multi-Stakeholder Networks": 0
    }

    # Define keyword checks for each theme
    checks = {
        "UN System Integration": ["un", "unesco", "who", "fao"],
        "Regional Collaboration": ["regional", "westpac", "iocAfrica"],
        "Scientific Partnerships": ["science", "research", "ices"],
        "Regulatory Alignment": ["regulation", "standards", "imo"],
        "Environmental Cooperation": ["environment", "unep", "cbd"],
        "Economic Linkages": ["trade", "economic", "unctad"],
        "Multi-Stakeholder Networks": ["stakeholder", "partnership", "global compact"]
    }

    # Combine the cleaned text and summary for analysis
    cleaned_lower = cleaned_text.lower()
    summary_lower = summary.lower()
    combined_text = cleaned_lower + " " + summary_lower

    # Score based on keyword checks
    for theme, keywords in checks.items():
        if any(keyword in combined_text for keyword in keywords):
            theme_scores[theme] += 1  # Increment score for matching keyword

    # Assign themes based on score threshold (e.g., >= 1)
    for theme, score in theme_scores.items():
        if score >= 1:  # Any theme with a score >= 1 will be added
            themes.add(theme)

    # Ensure at least one theme is assigned
    if not themes:
        themes.add("Multi-Stakeholder Networks")  # Default to this if no themes are matched
    
    return sorted(themes)  # Return sorted list for consistency

# Step 4: Constructing the Table
def process_column_updated(df: pd.DataFrame, column_name: str, output_file: str = "updated_inter_institutional_relationships.csv"):
    """Process the specified column in the DataFrame and output to CSV with new themes."""
    if column_name not in df.columns or "Institutions" not in df.columns:
        raise ValueError("DataFrame must contain 'Institution' and the specified column.")
    
    table = []
    for index, row in df.iterrows():
        institution = row["Institutions"]
        raw_text = row[column_name]
        
        cleaned = clean_data(raw_text)
        summary = summarize_data(cleaned)
        
        # Use the updated assign_themes function here
        themes = assign_themes_updated(cleaned, summary)
        
        table.append({
            "Institutions": institution,
            "Cleaned Defined Inter-Institutional Relationship": cleaned,
            "Summarized Defined Inter-Institutional Relationship": summary,
            "Themes": ";".join(themes)
        })
    
    with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ["Institutions", "Cleaned Defined Inter-Institutional Relationship", "Summarized Defined Inter-Institutional Relationship", "Themes"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        for row in table:
            writer.writerow(row)
    
    print(f"Processing complete. Output saved to {output_file}")


In [7]:
# Run the script with sample data
process_column_updated(df, "Defined inter-institutional Relationship")

Processing complete. Output saved to updated_inter_institutional_relationships.csv
